In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy
import boto3
import pyarrow
import pymysql
import mysql.connector
from bs4 import BeautifulSoup
from urllib.request import urlopen
from datetime import datetime, timezone, timedelta
from utils import *
# from decouple import config

In [31]:
year_stats = 2022
url = f"https://www.basketball-reference.com/leagues/NBA_{year_stats}_per_game.html"

html = urlopen(url)
# soup = BeautifulSoup(html, "html.parser")

In [32]:
import pickle

year_stats = 2022
url = f"https://www.basketball-reference.com/leagues/NBA_{year_stats}_per_game.html"
html = urlopen(url)

with open('new_test/tests/fixture_csvs/stats_html.html', 'wb') as fp:
    while True:
        chunk = html.read(1024)
        if not chunk:
            break
        fp.write(chunk)


In [37]:
with open('new_test/tests/fixture_csvs/stats_html.html', "rb") as fp:
    html1 = fp.read()

soup2 = BeautifulSoup(html1, "html.parser")

In [34]:
# with open('new_test/tests/fixture_csvs/stats_html', "rb") as fp:
#     html1 = pickle.load(fp)

with open('new_test/tests/fixture_csvs/stats_html.html', "rb") as fp:
    soup1 = BeautifulSoup(fp, "html.parser")
    
    # BeautifulSoup(fp.read(), "lxml")


# soup1 = BeautifulSoup(html1, "html.parser")
# soup1

In [18]:
aa = BeautifulSoup(html)

In [2]:
s3 = boto3.resource(
    service_name ='s3',
    region_name = os.environ.get('AWS_REGION'),
    aws_access_key_id = os.environ.get('AWS_KEY'),
    aws_secret_access_key = os.environ.get('AWS_SECRET')
)

In [3]:
for bucket in s3.buckets.all():
    print(bucket.name)

    b

jacobs-lambda-packages
jacobsbucket97


In [ ]:
def replace_all(text, dic):
  for i, j in dic.iteritems():
    text = text.replace(i, j)
  return text

In [26]:
prefixes_to_remove = [' Jr.', ' II', ' III', ' Sr.', ' IV']
# prefixes_to_remove = {' Jr.': "", ' II': "", ' III': "", ' Sr.': "", ' IV': ""}

def replace_all(dict, str):
    for key in dict:
        str = str.replace(key, dict[key])
    return str
# name = 'Gary Trent Jr.'

# new_name = name.replace(' Jr.', "")
# print(f"{new_name}")

# for i in prefixes_to_remove:
#     new_name2 = name.replace(i, "")
#     print(new_name2)

# for i in range(1, 100):
#     print(i * i)
print(str.replace('Gary Trent Jr.', ' Jr.', ""))

Gary Trent


In [20]:
print(replace_all('Gary Trent Jbbr.', {' Jr. ': ""}))

AttributeError: 'dict' object has no attribute 'replace'

In [10]:
class Comedian:
    def __init__(self, first_name, last_name, age):
        self.first_name = first_name
        self.last_name = last_name
        self.age = age

    def __str__(self):
        return f"{self.first_name} {self.last_name} is {self.age}."

    def __repr__(self):
        return f"{self.first_name} {self.last_name} is fuckin {self.age}. Surprise!"

bby1 = Comedian('Jacob', 'Otis', 74)
bby1
print(bby1) # the print object
bby1        # the literal representation of the object when we call it.

Jacob Otis is 74.


Jacob Otis is fuckin 74. Surprise!

In [27]:
name_list = ['Gary Trent Jr.', 'Marcus Morris Sr.']
df = pd.DataFrame(name_list)
df = df.rename(columns = {df.columns[0]: 'player'})

for i in prefixes_to_remove:
    df['player'] = df['player'].str.replace(i, "", regex = True)

In [7]:
# READ a file directly from s3.  no download.
obj = s3.Bucket('mygamelogsbucket').Object('bby.csv').get()
s3_gamelogs = pd.read_csv(obj['Body'], index_col = 0) # removes the useless index column

In [4]:
### #SQL DATABASE STUFF
mydb = sqlalchemy.create_engine('mysql+mysqlconnector://' + os.environ.get('RDS_USER') + ':' + os.environ.get('RDS_PW') + '@' + os.environ.get('IP') + ':' + os.environ.get('PORT') + '/' + os.environ.get('RDS_DB'),
                     echo = False)


In [5]:
bb1 = pd.read_sql('SELECT * FROM aws_odds_df;', con = mydb)


In [6]:
cnx = mydb.raw_connection()

In [3]:
from datetime import datetime, timedelta
import pandas as pd
# df = pd.to_datetime('2021-11-19', format = '%Y-%m-%d').date() - timedelta(days = 1)
# print(df)
df = pd.to_datetime('2018-01-01T00:00:00+00:00')
df2 = pd.to_datetime(datetime.strftime(df, format = '%Y-%m-%d:%H:%M:%S'), format = '%Y-%m-%d:%H:%M:%S')
df3 = pd.to_datetime(datetime.strftime(pd.to_datetime('2018-01-01T00:00:00+00:00'), format = '%Y-%m-%d:%H:%M:%S'), format = '%Y-%m-%d:%H:%M:%S')

In [11]:
connection = mysql.connector.connect(host = os.environ.get('IP'),
                                     port = os.environ.get('PORT'),
                                      user = os.environ.get('USER'),
                                      password = os.environ.get('PW'),
                                      db = os.environ.get('DB'))


# connection

all_tables = pd.read_sql_query('SHOW TABLES FROM aws_database;', connection)

In [12]:
df1 = pd.read_sql('SELECT * FROM aws_gamelogs;', con = connection)
df2 = pd.read_sql_query('SELECT * FROM aws_gamelogs;', con = connection)

In [13]:
gamelogs = pd.read_sql_query('SELECT * FROM aws_gamelogs;', connection)

gamelogs_new = gamelogs.groupby(['Team', 'Type', 'GameID']).mean('PTS')

gamelogs_type_aggs = gamelogs[(gamelogs['Type'] != 'Play-In Tournament')] \
    .groupby(['Team', 'Type', 'GameID']) \
    .sum()['PTS'] \
    .groupby(['Team', 'Type']) \
    .mean().round(1) \
    .reset_index() \
    .rename(columns = {'PTS' : 'team_pts'}) \
    .sort_values('team_pts', ascending = False) \
    .assign(language = 'Python7-4')

In [39]:
top10 = gamelogs.groupby(['Player']) \
    .mean('PTS').round(1) \
    .nlargest(25, 'PTS')  \
    .reset_index() \
    .get(['Player', 'Season', 'PTS', 'PlusMinus'])
# top10_2 = top10[['Player', 'PTS']].filter(['PTS'] >= 25)
top10_filter = top10.query('Player == "Stephen Curry" | Player == "Bradley Beal"')
top10_ptsfilter = top10[['Player', 'PTS']].query('PTS >= 25')
# USE QUERY IT IS DISGUSTING.

In [52]:
twenty_pt_games = gamelogs.query('PTS >= 20 & Type == "Regular Season"') \
    .groupby('Player') \
    .count() \
    .reset_index() \
    .get(['Player', 'row_names']) \
    .sort_values('row_names', ascending = False)

twenty_pt_games2 = gamelogs.query('PTS >= 20') \
    .groupby('Player') \
    .count()

In [11]:
# gamelogs_type_aggs.to_sql(con = connection, name = "my_python_table", index = False, if_exists = "replace")
# doesnt work, cant use connection var

In [7]:
bb1.to_sql(con = mydb, name = "my_python_table", index = False, if_exists = "replace")


In [8]:
new_df = pd.read_sql_query('SELECT * FROM my_python_table;', con = mydb)

In [10]:
all_tables = pd.read_sql_query('SHOW TABLES FROM jacob_db;', cnx)
all_tables1 = pd.read_sql_query('SHOW TABLES FROM jacob_db;', mydb)
print(all_tables)
print(all_tables1)

  Tables_in_jacob_db
0        aws_odds_df
1    my_python_table
  Tables_in_jacob_db
0        aws_odds_df
1    my_python_table


In [14]:
# i can read sql with both mysql connector and sqlalchemy engine
# but can only write to rds with sqlalchemy engine

In [15]:
# start_time = datetime.utcnow()
# my_time = datetime.now().time()
# current_datetime = datetime.today().strftime("%Y-%m-%d %H:%M:%S %p %Z")
# current_date = datetime.now().date()
# print(current_date)
# print(current_datetime)
# print('starting function at', start_time)
yesterday = datetime.now().date() - timedelta(1)

In [14]:
# top 15 scorers from most recent games
df = pd.read_sql_query('SELECT * FROM aws_gamelogs;', con = mydb)

df['Date'] = pd.to_datetime(df['Date']).dt.date

In [17]:
min(df['Date'])
max(df['Date'])

datetime.date(2021, 7, 6)

In [20]:
top10_yesterday = df[df['Date'] == max(df['Date'])] \
    .nlargest(10, 'PTS') \
    .sort_values('PTS', ascending = False)
# top10_yesterday = df2[['Player', 'Date', 'Outcome', 'Team', 'PTS']]

In [25]:
df.groupby('Team').sum('PTS')

KeyError: ('Team', 'PTS')